In [1]:
import pandas as pd
import glob
import numpy as np

In [2]:
df_master=pd.read_csv('/home/affine/Downloads/Deep_Learning/demo/demo/FSL/Document Classification/df_master.csv')

In [3]:
df_master.head()

,File_name,text,label,newlabel,HCFA,HCFASB,MEDICARE,UB,USB
0,../UB/UB2.tiff,"Am\v:mllW ' mun "" ‘mmmMmmm "" 1n «m pm... mm ...",UB,"[0, 0, 0, 1, 0]",0,0,0,1,0
1,../UB/UB1.tiff,yB04 RED ...,UB,"[0, 0, 0, 1, 0]",0,0,0,1,0
2,../UB/UB7.tiff,mm mm“: mm m ‘ 95m,UB,"[0, 0, 0, 1, 0]",0,0,0,1,0
3,../UB/UB8.tiff,_E_h wooowms mam-com ——6 “WWW“ 5554876789 “W ...,UB,"[0, 0, 0, 1, 0]",0,0,0,1,0
4,../UB/UB3.tiff,' m alum. mm m Ecwxmom Anrmwu w. 55551114 Sa...,UB,"[0, 0, 0, 1, 0]",0,0,0,1,0


In [21]:

import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, concatenate
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [9]:
EMBEDDING_FILE='/home/affine/Downloads/Deep_Learning/demo/demo/Toxic_Comment/glove.6B/glove.6B.300d.txt'

embed_size = 300 # how big is each word vector
max_features = 814 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
list_classes = ["HCFA", "HCFASB","MEDICARE","UB","USB"]

#Read in our data and replace missing values
train_X,test_X,train_Y,test_Y=train_test_split(df_master[['File_name','text']],df_master[list_classes], test_size=0.33, random_state=42)
# print(train_X)
list_sentences_train = train_X['text'].fillna("_na_").values

list_sentences_test = test_X['text'].fillna("_na_").values



In [10]:
y=train_Y.values
# y

In [11]:
#Preprocessing
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)


In [12]:
#Read the glove word vectors (space delimited strings) into a dictionary from word->vector.
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))


In [13]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std


(-0.0039050116, 0.38177028)

In [14]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [15]:
print(embedding_matrix.shape)

(814, 300)


In [22]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)

x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 

# x = GlobalMaxPool1D()(x)
x = Dense(300, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(5, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
adam=Adam(lr=0.0001,beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam , metrics=['accuracy'])


In [23]:
model.fit(X_t, y, batch_size=1, epochs=10, validation_split=0.25);

Train on 12 samples, validate on 4 samples
Epoch 1/10
12/12 [==============================] - 5s 421ms/step - loss: 1.6267 - acc: 0.1667 - val_loss: 1.6024 - val_acc: 0.5000
Epoch 2/10
12/12 [==============================] - 3s 289ms/step - loss: 1.4201 - acc: 0.5000 - val_loss: 1.5445 - val_acc: 0.5000
Epoch 3/10
12/12 [==============================] - 4s 295ms/step - loss: 1.2497 - acc: 0.5000 - val_loss: 1.5712 - val_acc: 0.5000
Epoch 4/10
12/12 [==============================] - 3s 286ms/step - loss: 1.1836 - acc: 0.5833 - val_loss: 1.5900 - val_acc: 0.5000
Epoch 5/10
12/12 [==============================] - 3s 284ms/step - loss: 1.1209 - acc: 0.5000 - val_loss: 1.6285 - val_acc: 0.5000
Epoch 6/10
12/12 [==============================] - 3s 286ms/step - loss: 1.0154 - acc: 0.5833 - val_loss: 1.6643 - val_acc: 0.5000
Epoch 7/10
12/12 [==============================] - 3s 285ms/step - loss: 0.9073 - acc: 0.8333 - val_loss: 1.7674 - val_acc: 0.5000
Epoch 8/10
12/12 [===============

In [40]:
# from keras.models import load_model

# model.save('LSTM_text_v4.h5')
# # model=load_model('LSTM_text_v1.h5')
# model.save_weights('weights_LSTM_text_v4.hdf5')
# # weights=model.load_weights('weights_LSTM_text_v1.hdf5')

In [24]:
y_test = model.predict([X_te], batch_size=1, verbose=1)

9/9 [==============================] - 1s 70ms/step


In [25]:
print("predicted:")
print(y_test)

predicted:
[[ 0.12241108  0.12036531  0.16311611  0.44924885  0.14485864]
 [ 0.07990497  0.2105297   0.13049746  0.24399625  0.33507162]
 [ 0.31461766  0.01159405  0.00771482  0.65575504  0.0103185 ]
 [ 0.22545519  0.01441539  0.01113502  0.736624    0.01237041]
 [ 0.26018515  0.0264395   0.01979118  0.67219537  0.0213887 ]
 [ 0.05554366  0.22525698  0.08471851  0.20941454  0.42506626]
 [ 0.55358201  0.01130235  0.0055436   0.41755742  0.01201465]
 [ 0.05866214  0.06253251  0.08933645  0.7093429   0.08012613]
 [ 0.24785516  0.01564858  0.01279948  0.71091479  0.0127821 ]]


In [26]:
print("Actual")
print(test_Y)

Actual
    HCFA  HCFASB  MEDICARE  UB  USB
8      1       0         0   0    0
16     1       0         0   0    0
0      0       0         0   1    0
23     0       0         1   0    0
11     1       0         0   0    0
9      1       0         0   0    0
13     1       0         0   0    0
1      0       0         0   1    0
22     0       0         1   0    0


In [27]:
y_pred=np.array(y_test)
y_pred1=y_pred.argmax(1)
print('test_pred  ',y_pred1)
y_actual=test_Y.values
y_actual1=y_actual.argmax(1)
print('test_actual',y_actual1)

test_pred   [3 4 3 3 3 4 0 3 3]
test_actual [0 0 3 2 0 0 0 3 2]


In [28]:
from sklearn.metrics import confusion_matrix
conf_mat=confusion_matrix(y_actual1,y_pred1)
print(conf_mat)

[[1 0 2 2]
 [0 0 2 0]
 [0 0 2 0]
 [0 0 0 0]]


In [29]:
from sklearn.metrics import classification_report
target_names=["HCFA", "HCFASB","MEDICARE","UB","USB"]
print(classification_report(y_actual1,y_pred1,target_names=target_names))

             precision    recall  f1-score   support

       HCFA       1.00      0.20      0.33         5
     HCFASB       0.00      0.00      0.00         2
   MEDICARE       0.33      1.00      0.50         2
         UB       0.00      0.00      0.00         0

avg / total       0.63      0.33      0.30         9



/home/affine/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 4, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/home/affine/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/affine/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
df1=pd.DataFrame(y_test)
df1.columns=list_classes
# print(df1)
df2=pd.DataFrame((test_Y))
df2.columns=list_classes
# print(df2)
df3=test_X
# print(df3)
df4=pd.concat([df3,df2], axis=1)
df4=pd.DataFrame(np.array(df4))
# print(df4)
df=pd.concat([df4,df1], axis=1)

In [31]:
df.columns=['File_name','text',"HCFA", "HCFASB","MEDICARE","UB","USB","HCFA_pred", "HCFASB_pred","MEDICARE_pred","UB_pred","USB_pred"]
df

,File_name,text,HCFA,HCFASB,MEDICARE,UB,USB,HCFA_pred,HCFASB_pred,MEDICARE_pred,UB_pred,USB_pred
0,../HCFA/HCFA19.tiff,PLEASE no not STAPLE m ms AREA 'Vnmfa '1 a...,1,0,0,0,0,0.122411,0.120365,0.163116,0.449249,0.144859
1,../HCFA/HCFA20.tiff,"was)»- EEK"" Eﬁ ""an"" Immune: cum ram mammmm ...",1,0,0,0,0,0.079905,0.210530,0.130497,0.243996,0.335072
2,../UB/UB2.tiff,"Am\v:mllW ' mun "" ‘mmmMmmm "" 1n «m pm... mm ...",0,0,0,1,0,0.314618,0.011594,0.007715,0.655755,0.010319
3,../MEDICARE/MEDICARE4.tiff,"r [m mmwmm‘mm WWW mamw mm"", 999999999 WWW 905m...",0,0,1,0,0,0.225455,0.014415,0.011135,0.736624,0.012370
4,../HCFA/HCFA13.tiff,Hum-Ina PO Eux new Lexmgknn Kv 4am 245m x eea...,1,0,0,0,0,0.260185,0.026439,0.019791,0.672195,0.021389
5,../HCFA/HCFA11.tiff,mmhw‘M—A EIICF‘E El HEALTH .INsunANcE CLAIM ...,1,0,0,0,0,0.055544,0.225257,0.084719,0.209415,0.425066
6,../HCFA/HCFA14.tiff,Nznlummueecwusnln BMW.- DL.-EWEV Dwa...,1,0,0,0,0,0.553582,0.011302,0.005544,0.417557,0.012015
7,../UB/UB1.tiff,yB04 RED ...,0,0,0,1,0,0.058662,0.062533,0.089336,0.709343,0.080126
8,../MEDICARE/MEDICARE3.tiff,M mamaalmcm- 2.1mm WWW maeome mm 999999999 Mm...,0,0,1,0,0,0.247855,0.015649,0.012799,0.710915,0.012782


In [94]:
df.to_csv('test_FSL.csv',index=False)

In [32]:
#Train Prediction
y_train1 = model.predict([X_t], batch_size=1, verbose=1)

16/16 [==============================] - 1s 43ms/step


In [33]:
y_pred_train=np.array(y_train1)
y_pred_train1=y_pred_train.argmax(1)
print('train_pred  ',y_pred_train1)
y_actual_train=train_Y.values
y_actual_train1=y_actual_train.argmax(1)
print('train_actual',y_actual_train1)

train_pred   [3 3 0 0 3 3 2 0 2 4 4 3 0 4 3 3]
train_actual [3 3 0 0 3 3 2 0 2 4 1 3 0 0 2 3]


In [34]:
from sklearn.metrics import confusion_matrix
conf_mat_train=confusion_matrix(y_actual_train1,y_pred_train1)
print(conf_mat_train)

[[4 0 0 0 1]
 [0 0 0 0 1]
 [0 0 2 1 0]
 [0 0 0 6 0]
 [0 0 0 0 1]]


In [35]:
#precision, recall and f1-score
from sklearn.metrics import classification_report
target_names=["HCFA", "HCFASB","MEDICARE","UB","USB"]
print(classification_report(y_actual_train1,y_pred_train1,target_names=target_names))

             precision    recall  f1-score   support

       HCFA       1.00      0.80      0.89         5
     HCFASB       0.00      0.00      0.00         1
   MEDICARE       1.00      0.67      0.80         3
         UB       0.86      1.00      0.92         6
        USB       0.33      1.00      0.50         1

avg / total       0.84      0.81      0.81        16



/home/affine/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
df11=pd.DataFrame(y_train1)
df11.columns=list_classes
# print(df1)
df21=pd.DataFrame((train_Y))
df21.columns=list_classes
# print(df2)
df31=train_X
# print(df3)
df41=pd.concat([df31,df21], axis=1)
df41=pd.DataFrame(np.array(df41))
# print(df4)
df_1=pd.concat([df41,df11], axis=1)

In [37]:
df_1.columns=['File_name','text',"HCFA", "HCFASB","MEDICARE","UB","USB","HCFA_pred", "HCFASB_pred","MEDICARE_pred","UB_pred","USB_pred"]
df_1

,File_name,text,HCFA,HCFASB,MEDICARE,UB,USB,HCFA_pred,HCFASB_pred,MEDICARE_pred,UB_pred,USB_pred
0,../UB/UB5.tiff,QB04 UMMARV ...,0,0,0,1,0,0.065586,0.055352,0.089938,0.714347,0.074777
1,../UB/UB7.tiff,mm mm“: mm m ‘ 95m,0,0,0,1,0,0.339501,0.008238,0.004095,0.638790,0.009377
2,../HCFA/HCFA15.tiff,lac/Is or Mama :2» mm mm W was NEALIN mm: su...,1,0,0,0,0,0.376086,0.105514,0.172132,0.238941,0.107327
3,../HCFA/HCFA17.tiff,—_p—nq ...,1,0,0,0,0,0.724554,0.008968,0.003771,0.251686,0.011021
4,../UB/UB8.tiff,_E_h wooowms mam-com ——6 “WWW“ 5554876789 “W ...,0,0,0,1,0,0.094292,0.069725,0.114426,0.641761,0.079796
5,../UB/UB3.tiff,' m alum. mm m Ecwxmom Anrmwu w. 55551114 Sa...,0,0,0,1,0,0.195531,0.013994,0.010158,0.768522,0.011794
6,../MEDICARE/MEDICARE5.tiff,______________________________________________...,0,0,1,0,0,0.124295,0.143946,0.400988,0.213371,0.117400
7,../HCFA/HCFA18.tiff,g mm muAm cum rm -,1,0,0,0,0,0.688816,0.008324,0.003510,0.289060,0.010290
8,../MEDICARE/MEDICARE1.tiff,a.” mmm FL MEDICARE 5:2 mvswsuus we p 0 am ...,0,0,1,0,0,0.134980,0.093925,0.385013,0.302963,0.083119
9,../USB/USB1.tiff,".1500, HEALYH msumuc: CLAIM FORM 7 m m:- . . u...",0,0,0,0,1,0.056982,0.263427,0.074430,0.147014,0.458147


In [93]:
df_1.to_csv('train_FSL.csv',index=False)